In [1]:
import pandas as pd
import numpy as np
import rasterio
import os


# 读取文件路径
vpd_path = r'G:\paper01\era5\VPD_mon_Clip'
sm_path = r'G:\paper01\era5\sm_layer2_mon_res\1'

# 设置工作空间为指定的文件夹
workspace_path = 'G:'  # 假设您想将工作空间设在这个路径
os.chdir(workspace_path)

In [2]:
# 定义日期范围
date_range = pd.date_range('1982-01-01', '2018-12-31', freq='M')

# 栅格的大小为360x720 (VPD) 和 361x720 (SM)
# 初始化存储数组，增加一个时间维度（即date_range的长度）
northern_hemisphere_vpd = np.empty((len(date_range), 180, 720), dtype=np.float32)
southern_hemisphere_vpd = np.empty((len(date_range), 180, 720), dtype=np.float32)
northern_hemisphere_sm = np.empty((len(date_range), 180, 720), dtype=np.float32)
southern_hemisphere_sm = np.empty((len(date_range), 180, 720), dtype=np.float32)

# 将初始数组填充为NaN（防止后续计算出错）
northern_hemisphere_vpd.fill(np.nan)
southern_hemisphere_vpd.fill(np.nan)
northern_hemisphere_sm.fill(np.nan)
southern_hemisphere_sm.fill(np.nan)

# 定义北半球和南半球对应的月份
northern_months = [4, 5, 6, 7, 8, 9]  
southern_months = [10, 11, 12, 1, 2, 3]  

# VPD和SM的纬度行划分，注意行数不同
vpd_northern_hemisphere_rows = slice(0, 180)  
vpd_southern_hemisphere_rows = slice(180, 360)  
sm_northern_hemisphere_rows = slice(0, 180)  
sm_southern_hemisphere_rows = slice(180, 360) 

百分位数和标准差

In [3]:
def calculate_monthly_90th_percentile(vpd_path,  num_rows, num_cols):
    # 初始化数组
    northern_vpd_90th_percentile = np.zeros((12, num_rows, num_cols))
    southern_vpd_90th_percentile = np.zeros((12, num_rows, num_cols))
    nor_vpd_std = np.zeros((12, num_rows, num_cols))
    sou_vpd_std = np.zeros((12, num_rows, num_cols))

    # 组织数据：创建一个字典来收集每个月份的所有年份的数据
    northern_hemisphere_vpd = {month: [] for month in northern_months}
    southern_hemisphere_vpd = {month: [] for month in southern_months}

    # 遍历每年每月，收集数据
    for date in date_range:
        month = date.month
        year = date.year
        # 读取VPD数据
        vpd_file_path = f"{vpd_path}/VPD_X{year}.{month:02d}.tif"
        if os.path.exists(vpd_file_path):
            with rasterio.open(vpd_file_path) as vpd_src:
                vpd_data = vpd_src.read(1)  # 读取整个全球的VPD数据
                vpd_data[vpd_data < 0] = np.nan  # 将非常小的值替换为 NaN，避免溢出
                
                # 处理北半球4-9月数据
                if month in northern_months:
                    northern_hemisphere_vpd[month].append(vpd_data[vpd_northern_hemisphere_rows, :])
                
                # 处理南半球10-3月数据
                if month in southern_months:
                    southern_hemisphere_vpd[month].append(vpd_data[vpd_southern_hemisphere_rows, :])

    # 计算每月的90百分位数
    for month in range(1, 13):
        if month in northern_months:
            i = month - 1  # 转换月份到索引
            if northern_hemisphere_vpd[month]:  # 确保有数据
                data_stack_north = np.stack(northern_hemisphere_vpd[month])
                northern_vpd_90th_percentile[i] = np.percentile(data_stack_north, 90, axis=0)
                nor_vpd_std[i] = np.nanstd(data_stack_north, axis=0)
        if month in southern_months:
            i = month - 1
            if southern_hemisphere_vpd[month]:
                data_stack_south = np.stack(southern_hemisphere_vpd[month])
                southern_vpd_90th_percentile[i] = np.percentile(data_stack_south, 90, axis=0)
                sou_vpd_std[i] = np.nanstd(data_stack_south, axis=0)
    return northern_vpd_90th_percentile, southern_vpd_90th_percentile, nor_vpd_std, sou_vpd_std


num_rows = 180  
num_cols = 720  

# 调用函数
northern_vpd_90th_percentile, southern_vpd_90th_percentile, nor_vpd_std, sou_vpd_std = calculate_monthly_90th_percentile(vpd_path, num_rows, num_cols)


d:\anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [4]:
vpd_90th = np.concatenate((northern_vpd_90th_percentile, southern_vpd_90th_percentile), axis=1)  # 沿行（垂直）拼接
vpd_std = np.concatenate((nor_vpd_std, sou_vpd_std), axis=1)  # 沿行（垂直）拼接


In [5]:
def calculate_monthly_10th_percentile(sm_path,  num_rows, num_cols):
    # 初始化用来存储每月百分位数的数组
    northern_sm_10th_percentile = np.zeros((12, num_rows, num_cols))
    southern_sm_10th_percentile = np.zeros((12, num_rows, num_cols))
    nor_sm_std = np.zeros((12, num_rows, num_cols))
    sou_sm_std = np.zeros((12, num_rows, num_cols))

    # 组织数据：创建一个字典来收集每个月份的所有年份的数据
    northern_hemisphere_sm = {month: [] for month in northern_months}
    southern_hemisphere_sm = {month: [] for month in southern_months}

    # 遍历每年每月，收集数据
    for date in date_range:
        month = date.month
        year = date.year
        # 读取SM数据
        sm_file_path = f"{sm_path}/sm_{year}{month:02d}.tif"
        if os.path.exists(sm_file_path):
            with rasterio.open(sm_file_path) as sm_src:
                sm_data = sm_src.read(1)  # 读取整个全球的VPD数据
                sm_data[sm_data < 0] = np.nan  # 将非常小的值替换为 NaN，避免溢出
                
                # 处理北半球4-9月数据
                if month in northern_months:
                    northern_hemisphere_sm[month].append(sm_data[sm_northern_hemisphere_rows, :])
                
                # 处理南半球10-3月数据
                if month in southern_months:
                    southern_hemisphere_sm[month].append(sm_data[sm_southern_hemisphere_rows, :])

    # 计算每月的10百分位数和标准差
    for month in range(1, 13):
        if month in northern_months:
            i = month - 1  # 转换月份到索引
            if northern_hemisphere_sm[month]:  # 确保有数据
                data_stack_north = np.stack(northern_hemisphere_sm[month])
                northern_sm_10th_percentile[i] = np.percentile(data_stack_north, 10, axis=0)
                nor_sm_std[i] = np.nanstd(data_stack_north, axis=0)
        if month in southern_months:
            i = month - 1
            if southern_hemisphere_sm[month]:
                data_stack_south = np.stack(southern_hemisphere_sm[month])
                southern_sm_10th_percentile[i] = np.percentile(data_stack_south, 10, axis=0)
                sou_sm_std[i] = np.nanstd(data_stack_south, axis=0)
    return northern_sm_10th_percentile, southern_sm_10th_percentile, nor_sm_std, sou_sm_std


num_rows = 180  
num_cols = 720  

# 调用函数
northern_sm_10th_percentile, southern_sm_10th_percentile, nor_sm_std, sou_sm_std = calculate_monthly_10th_percentile(sm_path, num_rows, num_cols)


In [6]:
sm_10th = np.concatenate((northern_sm_10th_percentile, southern_sm_10th_percentile), axis=1)  # 沿行（垂直）拼接
sm_std = np.concatenate((nor_sm_std, sou_sm_std), axis=1)  # 沿行（垂直）拼接


In [7]:
vpd_data = np.full((444, 360, 720), np.nan, dtype=np.float32)
sm_data = np.full((444, 360, 720), np.nan, dtype=np.float32)

# 读取VPD和SM数据
for idx, date in enumerate(date_range):
    month = date.month
    year = date.year
    # 读取VPD数据
    vpd_file_path = f"{vpd_path}/VPD_X{year}.{month:02d}.tif"
    if os.path.exists(vpd_file_path):
        with rasterio.open(vpd_file_path) as vpd_src:
            data = vpd_src.read(1)  # 读取整个全球的VPD数据
            data = data[:-1, :]  # 强制裁剪为 (360, 720)
            data[data <= 0] = np.nan
            vpd_data[idx] = data

    # 读取SM数据
    sm_file_path = f"{sm_path}/sm_{year}{month:02d}.tif"
    if os.path.exists(sm_file_path):
        with rasterio.open(sm_file_path) as sm_src:
            data1 = sm_src.read(1)  # 读取整个全球的VPD数据
            data1[data1 <= 0] = np.nan
            sm_data[idx] = data1

           

In [8]:
vpd_90th[vpd_90th <=0] = np.nan
vpd_std[vpd_std <=0] = np.nan
sm_10th[sm_10th <=0] = np.nan
sm_std[sm_std <=0] = np.nan

In [ ]:
print(avg_intensity_windows.shape)

In [ ]:
import matplotlib.pyplot as plt

# 确保 NaN 值不会影响颜色映射
plt.imshow(sm_std[1,:,:], cmap='viridis')
plt.colorbar(label='')  # 添加颜色条
plt.title('vpd1982-2018)')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()


In [ ]:
print(northern_vpd_90th_percentile.shape)

In [12]:
import numpy as np

def find_global_drought_events(vpd_data, sm_data, vpd_90th, sm_10th):
    """
    vpd_data, sm_data: (444, 360, 720)
    vpd_90th, sm_10th: (12, 360, 720)
    """
    n_months, n_lat, n_lon = sm_data.shape
    total_years = 37
    months_per_year = 12
    event = np.zeros_like(sm_data, dtype=bool)

    for month_index in range(n_months):
        month = month_index % 12 + 1
        percentile_index = month - 1

        # 北半球：lat 0–179，月 4–9
        if month in [4, 5, 6, 7, 8, 9]:
            event[month_index, 0:180, :] = (
                (sm_data[month_index, 0:180, :] < sm_10th[percentile_index, 0:180, :])
            )

        # 南半球：lat 180–359，月 1–3 和 10–12
        elif month in [1, 2, 3, 10, 11, 12]:
            event[month_index, 180:360, :] = (
                (sm_data[month_index, 180:360, :] < sm_10th[percentile_index, 180:360, :])
            )

        # 其余月份：默认保持 False（即非生长季）

    # 初始化滑动窗口结果
    window_size = 5
    num_windows = total_years - window_size + 1
    avg_duration_windows = np.zeros((num_windows, n_lat, n_lon), dtype=np.float32)
    frequency_windows = np.zeros((num_windows, n_lat, n_lon), dtype=np.float32)

    for w in range(num_windows):
        start_index = w * months_per_year
        end_index = start_index + (window_size * months_per_year)
        event_window = event[start_index:end_index]

        for i in range(n_lat):
            for j in range(n_lon):
                grid_event = event_window[:, i, j]
                current_duration = 0
                duration_sum = 0
                event_count = 0

                for m in range(grid_event.shape[0]):
                    if grid_event[m]:
                        current_duration += 1
                    else:
                        if current_duration > 0:
                            duration_sum += current_duration
                            event_count += 1
                            current_duration = 0
                if current_duration > 0:
                    duration_sum += current_duration
                    event_count += 1

                if event_count > 0:
                    avg_duration_windows[w, i, j] = duration_sum / event_count
                    frequency_windows[w, i, j] = event_count / window_size

    return avg_duration_windows, frequency_windows


In [13]:
avg_duration_windows, frequency_windows = find_global_drought_events(vpd_data, sm_data, vpd_90th, sm_10th)


In [14]:
avg_duration_windows[avg_duration_windows<=0] = np.nan
frequency_windows[frequency_windows<=0] = np.nan

In [15]:
# 计算所有像元的平均值，得到 33 个数
avg_duration_mean = np.nanmean(avg_duration_windows, axis=(1, 2))
frequency_mean = np.nanmean(frequency_windows, axis=(1, 2))

In [17]:
print(avg_duration_mean)

[1.9357188 2.083523  2.1067169 2.1151545 2.07902   1.8276945 1.6998461
 1.5951656 1.6020445 1.5835079 1.5636717 1.5651947 1.558116  1.5199639
 1.8784153 2.001328  2.0317156 2.0476801 2.053535  1.879194  1.750644
 1.6855471 1.6600842 1.6355474 1.9224492 2.068875  2.1144044 2.1370935
 2.1231534 2.145476  2.0961423 2.0479937 2.0209792]


In [18]:
import numpy as np

def find_intensity(vpd_data, sm_data, vpd_90th, sm_10th, vpd_std, sm_std):
    n_months, n_lat, n_lon = sm_data.shape
    total_years = 37
    months_per_year = 12
    event = np.zeros_like(sm_data, dtype=bool)

    for month_index in range(n_months):
        month = month_index % 12 + 1
        percentile_index = month - 1

        # 北半球：lat 0–179，月 4–9
        if month in [4, 5, 6, 7, 8, 9]:
            event[month_index, 0:180, :] = (
                (sm_data[month_index, 0:180, :] < sm_10th[percentile_index, 0:180, :])
            )

        # 南半球：lat 180–359，月 1–3 和 10–12
        elif month in [1, 2, 3, 10, 11, 12]:
            event[month_index, 180:360, :] = (
                (sm_data[month_index, 180:360, :] < sm_10th[percentile_index, 180:360, :])
            )

    # 初始化数组
    window_size = 5  # 5年窗口
    num_windows = total_years - window_size + 1  # 共33个窗口 (1982-1986, ..., 2014-2018)
    avg_intensity_windows = np.zeros((num_windows, event.shape[1], event.shape[2]), dtype=np.float32)
    total_intensity_windows = np.zeros((num_windows, event.shape[1], event.shape[2]), dtype=np.float32)

    for w in range(num_windows):
        start_index = w * months_per_year  
        end_index = start_index + (window_size * months_per_year)  
        event_window = event[start_index:end_index]

        # 提取当前窗口的数据
        event_window = event[start_index:end_index]
        sm_window = sm_data[start_index:end_index]
        # vpd_window = vpd_data[start_index:end_index]

        total_event_count_window = np.zeros(event.shape[1:], dtype=np.int32)
        total_intensity_window = np.zeros(event.shape[1:], dtype=np.float32)

        for i in range(event.shape[1]):
            for j in range(event.shape[2]):
                grid_event = event_window[:, i, j]

                current_duration = 0
                event_sm_sum = 0
                # event_vpd_sum = 0

                for month in range(grid_event.shape[0]):
                    month_in_year = month % 12  
                    if grid_event[month]:  
                        current_duration += 1
                        event_sm_sum += ((sm_window[month, i, j] - sm_10th[month_in_year, i, j]) / sm_std[month_in_year, i, j]) ** 2
                        # event_vpd_sum += ((vpd_window[month, i, j] - vpd_90th[month_in_year, i, j]) / vpd_std[month_in_year, i, j]) ** 2
                    else:
                        if current_duration > 0:
                            total_event_count_window[i, j] += 1
                            
                            intensity_val = np.sqrt((event_sm_sum ) / (2 * current_duration))
                            total_intensity_window[i, j] += intensity_val

                            current_duration = 0
                            event_sm_sum = 0
                            event_vpd_sum = 0   

                if current_duration > 0:
                    total_event_count_window[i, j] += 1
                    intensity_val = np.sqrt((event_sm_sum ) / (2 * current_duration))
                    total_intensity_window[i, j] += intensity_val

        # 计算平均强度，防止除0
        mask = total_event_count_window > 0
        avg_intensity_window = np.zeros(event.shape[1:], dtype=np.float32)
        avg_intensity_window[mask] = total_intensity_window[mask] / total_event_count_window[mask]

        # 存储结果
        avg_intensity_windows[w] = avg_intensity_window
        total_intensity_windows[w] = total_intensity_window

    return avg_intensity_windows, total_intensity_windows


In [19]:
avg_intensity_windows, total_intensity_windows = find_intensity(vpd_data, sm_data, vpd_90th, sm_10th,  vpd_std, sm_std)

In [20]:
avg_intensity_windows[avg_intensity_windows <= 0] = np.nan
total_intensity_windows[total_intensity_windows <= 0] = np.nan

In [21]:
# 计算所有像元的平均值，得到 33 个数
avg_intensity_mean = np.nanmean(avg_intensity_windows, axis=(1, 2))
total_intensity_mean = np.nanmean(total_intensity_windows, axis=(1, 2))

In [23]:
print(avg_intensity_mean)

[0.31179053 0.30478892 0.29558882 0.2949381  0.2960933  0.29417726
 0.3008587  0.30203322 0.2961296  0.29784995 0.29321536 0.30404463
 0.32016364 0.32503545 0.30944166 0.3002706  0.29313365 0.28029266
 0.28018343 0.2753276  0.28541812 0.29738083 0.31029132 0.31406492
 0.30919117 0.31175908 0.30373016 0.3032547  0.30422693 0.34114906
 0.3421072  0.339871   0.3431465 ]


In [24]:
# 创建 DataFrame
years = np.arange(1984, 2017)  # 1982-1986 -> 1984, ..., 2014-2018 -> 2016
df = pd.DataFrame({
    "Year": years,
    "CDE_Duration": avg_duration_mean,
    "CDE_Frequency": frequency_mean,
    "CDE_intensity": avg_intensity_mean,
})

# 保存为 CSV
df.to_csv(r"G:\paper01\M03图\SM_干旱特征5年滑窗.csv", index=False)
